In [8]:
import torch
import torch.nn as nn
import numpy as np

In [26]:
# input shape = B, 6 (or however many channels we want), len

# Conv1d(in_channels, out_channels, 
# kernel_size, stride=1, padding=0, dilation=1, 
# groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)



c_in = 6
num_classes = 3

channels = [12, 24, 48]
strides = [1, 1, 1]
paddings = [0, 0, 0]
dilations = [1, 1, 1]
ksizes = [3, 3, 3]
use_pool = True

pool_ksize = 2
pool_stride = 1
pool_dilation = 1
pool_padding = 0

linear_features = [48, 24, 12]

# torch.nn.Linear(in_features, out_features, bias=True, device=None, dtype=None)

model = nn.Sequential()

for layer_ind, (out_channels, ksize, stride, padding, dilation) in enumerate(zip(channels, ksizes, strides, paddings, dilations)):
    if layer_ind == 0:
        in_channels = c_in
    else:
        in_channels = channels[layer_ind - 1]
        
    model.append(
        nn.Conv1d(in_channels=in_channels, kernel_size=ksize, out_channels=out_channels, stride=stride, padding=padding, dilation=dilation)
    )
    model.append(
        nn.ReLU()
    )
    if use_pool:
        model.append(
            nn.MaxPool1d(kernel_size = pool_ksize, stride=pool_stride, padding=pool_padding, dilation=pool_dilation)
        )
        
model.append(
    nn.Flatten()
)
        
for lin_ind, lin_features in enumerate(linear_features):
    
    out_features = lin_features
    if lin_ind == 0:
        model.append(
            nn.LazyLinear(out_features)
        )
    else:
        in_features = linear_features[lin_ind - 1]
        
        model.append(
            nn.Linear(in_features = in_features, out_features = out_features)
        )
    model.append(
        nn.ReLU()
    )
        
model.append(
    nn.Linear(in_features = linear_features[-1], out_features = num_classes)
)

/home/oscar/Work/Studies/WASP/AutonomousSystems/activitty_recognition/venv/lib/python3.10/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Sequential(
  (0): Conv1d(6, 12, kernel_size=(3,), stride=(1,))
  (1): ReLU()
  (2): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  (3): Conv1d(12, 24, kernel_size=(3,), stride=(1,))
  (4): ReLU()
  (5): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  (6): Conv1d(24, 48, kernel_size=(3,), stride=(1,))
  (7): ReLU()
  (8): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  (9): Flatten(start_dim=1, end_dim=-1)
  (10): LazyLinear(in_features=0, out_features=48, bias=True)
  (11): ReLU()
  (12): Linear(in_features=48, out_features=24, bias=True)
  (13): ReLU()
  (14): Linear(in_features=24, out_features=12, bias=True)
  (15): ReLU()
  (16): Linear(in_features=12, out_features=3, bias=True)
)

In [28]:
test_data = torch.tensor(np.random.random((10,6,100)),dtype=torch.float32)
model(test_data).shape


RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x47568 and 4368x48)